In [1]:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/input/mri-predictor/tensorflow2/f10.96/1/my_cnn_model.h5')


2024-04-24 01:21:34.313733: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 01:21:34.313843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 01:21:34.436097: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img = cv2.resize(img, (256, 256)) 
            img_array = np.array(img)
            folder = input_folder.split("/")[-1]
            output_file_path = os.path.join(output_folder, f"{folder}{i}.npy")
            np.save(output_file_path, img_array)

jpg_to_npy_in_folders("/kaggle/input/alzheimersdisease5classdatasetadni/Alzheimers-ADNI/train", "/kaggle/working/train")

In [3]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

1101

In [4]:
ad = [i for i in files if "AD" in i]
cn = [i for i in files if "CN" in i]
emci = [i for i in files if "EMCI" in i]
lmci = [i for i in files if "LMCI" in i]
mci = [i for i in files if " MCI " in i]
print(len(ad), len(cn), len(emci), len(lmci), len(mci))

145 493 204 61 198


In [5]:
import pandas as pd
l = [[ad, 0], [cn, 1], [emci, 2], [lmci, 3], [mci, 4]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])


In [6]:
images = []
for i in df["Path"]:
    try:
        images.append(np.load(i))
    except:
        print(i)
images = np.array(images)

In [7]:
import pandas as pd
df1 = pd.read_csv("/kaggle/input/handwriting-data-to-detect-alzheimers-disease/data.csv")
df1["class"] = pd.Series([1 if i == 'P' else 0 for i in df1["class"]])
df1

,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,id_1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,1
1,id_2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,1
2,id_3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,1
3,id_4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,1
4,id_5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,id_170,2930,0.000010,241.736477,176.115957,1839,6439,0.253347,0.174663,208.926217,...,0.119152,0.020909,4.508709,2.233198,96,44545,1798.923336,247448.3108,80335,0
170,id_171,2140,0.000009,274.728964,234.495802,2053,8487,0.225537,0.174920,254.612383,...,0.174495,0.017640,4.685573,2.806888,84,37560,1725.619941,160664.6464,345835,0
171,id_172,3830,0.000008,151.536989,171.104693,1287,7352,0.165480,0.161058,161.320841,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,0
172,id_173,1760,0.000008,289.518195,196.411138,1674,6946,0.518937,0.202613,242.964666,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,0


In [8]:
import random

x = df1.sample(50)
x["class"].value_counts()

class
0    31
1    19
Name: count, dtype: int64

In [9]:
nonAlz = x[x["class"] == 0]
d = []
e = []
for i in range(len(df)):
    if df["Class"][i] == 1:
        d.append(df["Path"][i])
        e.append(df["Class"][i])

nonAlz.loc[:, "path"] = pd.Series(d)
nonAlz.loc[:, "mri-class"] = pd.Series(e)
nonAlz

/tmp/ipykernel_24/4233089575.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonAlz.loc[:, "path"] = pd.Series(d)
/tmp/ipykernel_24/4233089575.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonAlz.loc[:, "mri-class"] = pd.Series(e)


,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class,path,mri-class
130,id_131,520,0.000010,601.206645,109.926339,1680,4893,0.195066,0.119620,355.566492,...,4.036907,2.119121,75,25560,1291.958529,184854.10980,47680,0,/kaggle/working/train/Final CN JPEG469.npy,1
95,id_96,2175,0.000010,188.083515,122.606139,1421,7488,0.221660,0.126281,155.344827,...,10.061253,5.602909,64,22425,1942.814939,101171.61210,36915,0,/kaggle/working/train/Final CN JPEG419.npy,1
105,id_106,3990,0.000010,289.788098,180.020926,1348,7379,0.188276,0.147752,234.904512,...,4.997431,3.366661,90,31715,1007.996847,94731.67207,339675,0,/kaggle/working/train/Final CN JPEG377.npy,1
129,id_130,2805,0.000008,135.318129,125.990855,1035,5310,0.322051,0.144647,130.654492,...,3.329704,2.008075,60,39775,1561.145820,147074.63340,64825,0,/kaggle/working/train/Final CN JPEG421.npy,1
100,id_101,7770,0.000007,252.920870,241.011868,2076,5683,0.498263,0.193680,246.966369,...,6.278983,3.277159,58,33560,1840.851162,144392.44640,63130,0,/kaggle/working/train/Final CN JPEG18.npy,1
156,id_157,3315,0.000016,234.722662,138.589041,1766,8247,0.209320,0.147288,186.655852,...,5.005057,2.523930,71,39190,1814.252105,156000.41280,76330,0,/kaggle/working/train/Final CN JPEG194.npy,1
118,id_119,540,0.000007,557.998222,221.396848,1697,6343,0.231201,0.157785,389.697535,...,5.741075,2.535960,73,31590,1942.588003,97106.70791,55120,0,/kaggle/working/train/Final CN JPEG3.npy,1
155,id_156,1260,0.000008,339.614859,186.696258,1559,6988,0.204238,0.167571,263.155559,...,2.949977,2.481760,100,63430,1884.644490,155557.52800,117435,0,/kaggle/working/train/Final CN JPEG280.npy,1
103,id_104,4375,0.000002,90.667148,243.484916,1550,1137,0.313019,0.177252,167.076032,...,6.421632,4.540908,58,26125,1837.481914,118930.38750,47325,0,/kaggle/working/train/Final CN JPEG12.npy,1
166,id_167,8735,0.000012,321.354345,158.889068,1379,9171,1.692302,0.176674,240.121706,...,5.114071,2.788386,80,32190,1903.081702,93727.84079,341405,0,/kaggle/working/train/Final CN JPEG422.npy,1


In [10]:
Alz = x[x["class"] == 1]
d = []
e = []
for i in range(len(df)):
    if df["Class"][i] != 1:
        d.append(df["Path"][i])
        e.append(df["Class"][i])

Alz.loc[:, "path"] = pd.Series(d)
Alz.loc[:, "mri-class"] = pd.Series(e)

Alz

/tmp/ipykernel_24/1587798616.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Alz.loc[:, "path"] = pd.Series(d)
/tmp/ipykernel_24/1587798616.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Alz.loc[:, "mri-class"] = pd.Series(e)


,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class,path,mri-class
29,id_30,9880,0.000015,137.524031,56.248264,901,7768,0.368335,0.181916,96.886147,...,5.662866,3.432212,71,38285,1705.952331,157649.09800,90485,1,/kaggle/working/train/Final AD JPEG69.npy,0
66,id_67,465,0.000006,935.805821,497.851811,1992,6070,0.860722,0.316033,716.828816,...,3.585643,2.500616,134,71075,1530.794724,184497.75410,181970,1,/kaggle/working/train/Final AD JPEG74.npy,0
18,id_19,6365,0.000011,78.339161,118.660574,1425,7755,0.127188,0.119886,98.499867,...,3.498404,2.073031,119,59035,474.049462,26984.92666,177155,1,/kaggle/working/train/Final AD JPEG122.npy,0
4,id_5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,3.319036,1.680629,92,37285,1841.702561,158290.02550,72575,1,/kaggle/working/train/Final AD JPEG61.npy,0
47,id_48,4450,0.000009,105.993634,85.215193,1157,6167,0.186082,0.163422,95.604413,...,4.308988,2.928210,100,41865,1462.454915,182825.98810,92150,1,/kaggle/working/train/Final AD JPEG43.npy,0
36,id_37,12960,0.000019,154.094876,66.137920,1335,9809,0.303533,0.168974,110.116398,...,5.171081,2.851422,81,33010,1583.533172,165641.91900,96545,1,/kaggle/working/train/Final AD JPEG140.npy,0
32,id_33,4410,0.000011,170.472145,132.929749,1910,8502,0.222532,0.139789,151.700947,...,2.293897,1.274208,128,39780,1528.106586,196881.93230,87150,1,/kaggle/working/train/Final AD JPEG132.npy,0
49,id_50,670,0.000007,536.433411,405.992762,3009,8371,0.483585,0.273639,471.213087,...,4.286445,1.910254,80,40800,1394.883578,212422.33870,77170,1,/kaggle/working/train/Final AD JPEG101.npy,0
26,id_27,3365,0.000006,139.705985,179.776831,1777,6415,0.206496,0.208559,159.741408,...,1.809954,2.213869,180,74185,1632.469098,352981.85000,238895,1,/kaggle/working/train/Final AD JPEG96.npy,0
67,id_68,4100,0.000009,140.527147,299.585046,2085,6154,0.258524,0.196004,220.056097,...,2.849443,3.285650,74,35535,1852.897003,159429.77970,117710,1,/kaggle/working/train/Final AD JPEG127.npy,0


In [11]:
nonAlz_y_handwriting = np.zeros(nonAlz.shape[0])
nonAlz_y_mri = nonAlz["mri-class"]

Alz_y_handwriting = np.ones(Alz.shape[0])
Alz_y_mri = Alz["mri-class"]

In [12]:
import pickle

with open('/kaggle/input/handwriitng-classifier/other/claassifier-gb/1/gb_classifier_weights.pkl', 'rb') as file:
    handwriting_model = pickle.load(file)

In [13]:
import random
s = [nonAlz, Alz]
x = []
for i in range(10):
    r = random.choice(s)
    x.append(r.sample(1))

In [14]:
def predictor(data):
    inp = data.drop(["ID", "class", "path", "mri-class"], axis=1)
    handwriting_prediction = handwriting_model.predict(inp)
    if handwriting_prediction.any() == 1:
        img = np.load(data["path"].values[0])
        img = np.expand_dims(img, axis=0) 
        prediction = np.argmax(model.predict(img), axis=1)[0]
        print(f"Model predicted - You have alzheimer's, type {prediction}")
        print(f"Actual case - You have alzheimer's, type {data['mri-class'].values[0]}")
    else:
        print("Model - predicted - You dont have alzheimer's")
        print(f"Actual case - You {'have' if data['class'].any() == 1 else 'dont have'} alzheimer's")

In [15]:
for i in x:
    predictor(i)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Model predicted - You have alzheimer's, type 0
Actual case - You have alzheimer's, type 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


I0000 00:00:1713921723.460482      73 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Model predicted - You have alzheimer's, type 0
Actual case - You have alzheimer's, type 0
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Model predicted - You have alzheimer's, type 0
Actual case - You have alzheimer's, type 0
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Model predicted - You have alzheimer's, type 0
Actual case - You have alzheimer's, type 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Model predicted - You have alzheimer's, type 0
Actual case - You have alzheimer's, type 0
